# Data cleaning challenge day 1 - Handling missing values

Well, I've been meaning to start a more structured attack on building my Python knowledge. At the moment, I'm lost in the comfort of doing things in R. For a lot of purposes, I'll probably stick to it, however, being able to do everything (or most of the things) I do in R in Python would certainly have its advantages.

For example, at the moment our company website is built using Python on a Django framework. It would certainly be handy to be able to perform analyses, run reports and build dashboards directly on the backend of the website...

So, time to hit [Day 1 of Rachael's challenge][1], and cleaning the data is a good place to start.

[1]: https://www.kaggle.com/rtatman/data-cleaning-challenge-handling-missing-values/notebook

In [1]:
# import numpy and pandas

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# read in the San Francisco building permits data
sfPermits = pd.read_csv("C:/Users/Travel Your Style/Documents/IOD Course 22 June 20/DATA/Building_Permits.csv")

# set seed for reproducibility
np.random.seed(0)

C:\Users\Travel Your Style\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Look at a couple of rows from the sf_permits dataset. Do you notice any missing data?

In [2]:
sfPermits.sample(5)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
40553,201403039652,8,otc alterations permit,03/03/2014,3732,008,400,NaN,Clementina,St,...,NaN,NaN,1.0,constr type 1,NaN,6.0,South of Market,94103.0,"(37.780460571778164, -122.40450626524974)",1334094491645
169731,201510159735,3,additions alterations or repairs,10/15/2015,2609,028,79,NaN,Buena Vista,Tr,...,5.0,wood frame (5),5.0,wood frame (5),NaN,8.0,Castro/Upper Market,94117.0,"(37.76757916496494, -122.43793170417105)",1399356139170
19180,M409787,8,otc alterations permit,07/22/2013,4624,031,178,NaN,West Point,Rd,...,NaN,NaN,NaN,NaN,NaN,10.0,Bayview Hunters Point,94124.0,"(37.73524725436046, -122.38063828309745)",1311685491725
68047,201411191888,8,otc alterations permit,11/19/2014,0039,109,294,NaN,Francisco,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,North Beach,94133.0,"(37.805257822817126, -122.40998545760392)",1362881288870
64238,M527228,8,otc alterations permit,10/14/2014,1251,002,707,NaN,Cole,St,...,NaN,NaN,NaN,NaN,NaN,5.0,Haight Ashbury,94117.0,"(37.76836885973765, -122.45074431487859)",135886493776


Quite a few missing values visible already, and we've only looked at the first five rows of the dataset, cleaning will be required...

### Find out what percent of the sf_permit dataset is missing

In [3]:
# Calculate total number of cells in dataframe
totalCells = np.product(sfPermits.shape)

# Count number of missing values per column
missingCount = sfPermits.isnull().sum()

# Calculate total number of missing values
totalMissing = missingCount.sum()

# Calculate percentage of missing values
print("The SF Permits dataset contains", round(((totalMissing/totalCells) * 100), 2), "%", "missing values.")


The SF Permits dataset contains 26.26 % missing values.


### Look at the columns Street Number Suffix and Zipcode from the sf_permits datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?

In [4]:
missingCount[['Street Number Suffix', 'Zipcode']]


Street Number Suffix    196684
Zipcode                   1716
dtype: int64

Looks like a lot more missing values for street number suffix than zipcode. Let's check out the percentages:

In [5]:
print("Percent missing data in Street Number Suffix column =", (round(((missingCount['Street Number Suffix'] / sfPermits.shape[0]) * 100), 2)))
print("Percent missing data in Zipcode column =", (round(((missingCount['Zipcode'] / sfPermits.shape[0]) * 100), 2)))

Percent missing data in Street Number Suffix column = 98.89
Percent missing data in Zipcode column = 0.86


As every address has a Zipcode, it looks like the missing values for this column are due to the values not being recorded. For the Street Number Suffix column, it is likely very few properties will have a suffix to the number, I see a lot of 3s, 18s, 46s, but not nearly as many 36A or 18B, so it is likely that these are missing as they don't exist.

### Try removing all the rows from the sf_permits dataset that contain missing values. How many are left?

In [6]:
sfPermits.dropna()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID


If we drop all rows that contain a missing value, we greatly simplify our dataset. So simple, we can go for an early lunch. Every row contains at least one missing value (well, we know from our Street Number Suffix answer above that simply eliminating those gets rid of nearly 99% of our data), so we end up with a dataframe of column headers.

### Now try removing all the columns with empty values. Now how much of your data is left?

In [7]:
sfPermitsCleanCols = sfPermits.dropna(axis=1)
sfPermitsCleanCols.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Current Status,Current Status Date,Filed Date,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,Ellis,expired,12/21/2017,05/06/2015,1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,Geary,issued,08/03/2017,04/19/2016,1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,Pacific,withdrawn,09/26/2017,05/27/2016,1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,Pacific,complete,07/24/2017,11/07/2016,1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,issued,12/01/2017,11/28/2016,144548169992


In [8]:
print("Columns in original dataset: %d \n" % sfPermits.shape[1])
print("Columns with na's dropped: %d" % sfPermitsCleanCols.shape[1])

Columns in original dataset: 43 

Columns with na's dropped: 12


Well, that gives us a clean set of values, but we've sacrificed a lot of variables in the process...

### Your turn! Try replacing all the NaN's in the sf_permit data with the one that comes directly after it and then [replace all the reamining na's with 0]

In [9]:
imputeSfPermits = sfPermits.fillna(method='ffill', axis=0).fillna("0")

imputeSfPermits.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,0,Ellis,St,...,3.0,constr type 3,0,0,0,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,0,Geary,St,...,3.0,constr type 3,0,0,0,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,0,Pacific,Av,...,1.0,constr type 1,1,constr type 1,0,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,0,Pacific,Av,...,5.0,wood frame (5),5,wood frame (5),0,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,0,Market,St,...,3.0,constr type 3,5,wood frame (5),0,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


That's certainly a nicer way to do things, but still quite a simplistic method. For EDA and preliminary analysis it's a good way to get started, but choosing an imputation method based on the type of data in each column would be a logical next step.

Either way, coming from R, it's been a good exercise to start getting my Python more up to scratch; thanks Rachael!